In [1]:
import xarray as xr
from os.path import join

# 定义所有需要遍历的维度
ssps = ['historical', 'ssp126', 'ssp585']
crops = ['mai', 'soy', 'whe']
drought_vars = ['ssi', 'spei']
# var='ssi'
result_dict1 = {
    ssp: {
            crop: {} for crop in crops
    } for ssp in ssps
}
for ssp in ssps:
        for crop in crops:
            # 特殊处理小麦（合并冬小麦和春小麦）
            if crop == 'whe':
                try:
                    fp = r'D:\AAUDE\paper\paper8\data\resu'
                    area_path = join(r'D:\AAUDE\paper\paper8\data\area', f'spam2020_v1r0_global_H_wwh_R.nc')
                    area_wwh = xr.open_dataset(area_path)[list(xr.open_dataset(area_path).data_vars)[0]]
                    area_path = join(r'D:\AAUDE\paper\paper8\data\area', f'spam2020_v1r0_global_H_swh_R.nc')
                    area_swh = xr.open_dataset(area_path)[list(xr.open_dataset(area_path).data_vars)[0]]
                    season_wwh=xr.open_dataset(rf'D:\AAUDE\paper\paper8\data\season\wwh_{ssp}_planting_to_maturity_masks.nc').mask_planting_to_maturity
                    season_swh=xr.open_dataset(rf'D:\AAUDE\paper\paper8\data\season\swh_{ssp}_planting_to_maturity_masks.nc').mask_planting_to_maturity
                    ssi_data_wwh = xr.open_dataset(join(fp, f'ssi_scale2_wwh_{ssp}_median.nc'))['ssi'].where(season_wwh)
                    ssi_data_swh = xr.open_dataset(join(fp, f'ssi_scale2_swh_{ssp}_median.nc'))['ssi'].where(season_swh)
                    spei_data_swh = xr.open_dataset(join(fp, f'SPEI_scale2_swh_{ssp}_median.nc'))['spei'].where(season_swh)
                    spei_data_wwh = xr.open_dataset(join(fp, f'SPEI_scale2_wwh_{ssp}_median.nc'))['spei'].where(season_wwh)
                    yied_wwh= xr.open_dataset(join(fp, f'yield_wwh_{ssp}_median.nc'))[f'yield-wwh-noirr']
                    yied_swh= xr.open_dataset(join(fp, f'yield_swh_{ssp}_median.nc'))[f'yield-swh-noirr']
                    
                    yied_swh['time'] = ssi_data_wwh.time.groupby('time.year').first().values
                    yied_wwh['time'] = ssi_data_wwh.time.groupby('time.year').first().values
                    ssi_data_wwh=ssi_data_wwh[:,:,-len(spei_data_swh.time):]
                    ssi_data_swh=ssi_data_swh[:,:,-len(spei_data_swh.time):]
                    yied_wwh=yied_wwh[int(-len(spei_data_swh.time)/12):-1]
                    yied_swh=yied_swh[int(-len(spei_data_swh.time)/12):-1]
                    ssi_data_wwh = ssi_data_wwh.groupby('time.year').mean('time')[:,:,:-1].rename({'year': 'time'})
                    spei_data_wwh = spei_data_wwh.groupby('time.year').mean('time')[:,:,:-1].rename({'year': 'time'})
                    ssi_data_swh = ssi_data_swh.groupby('time.year').mean('time')[:,:,:-1].rename({'year': 'time'})
                    spei_data_swh = spei_data_swh.groupby('time.year').mean('time')[:,:,:-1].rename({'year': 'time'})
                    
                    yied=yied_wwh.where(area_wwh>0).fillna(0)+yied_swh.where(area_swh>0).fillna(0)
                    ssi=ssi_data_wwh.where(area_wwh>0).fillna(0)+ssi_data_swh.where(area_swh>0).fillna(0)
                    spei=spei_data_wwh.where(area_wwh>0).fillna(0)+spei_data_swh.where(area_swh>0).fillna(0)
                    result_dict1[ssp][crop]['yield'] =yied.where(yied!=0)
                    result_dict1[ssp][crop]['spei'] = spei.where(spei!=0)
                    result_dict1[ssp][crop]['ssi'] = ssi.where(ssi!=0)
                except FileNotFoundError:
                    print(f"Warning: 缺少小麦数据文件 for {var}-{ssp}")
                    continue
            else:
                try:
                    fp = r'D:\AAUDE\paper\paper8\data\resu'
                    area_path = join(r'D:\AAUDE\paper\paper8\data\area', f'spam2020_v1r0_global_H_{crop}_R.nc')
                    area = xr.open_dataset(area_path)[list(xr.open_dataset(area_path).data_vars)[0]]
                    season=xr.open_dataset(rf'D:\AAUDE\paper\paper8\data\season\{crop}_{ssp}_planting_to_maturity_masks.nc').mask_planting_to_maturity
                    ssi_data = xr.open_dataset(join(fp, f'ssi_scale2_{crop}_{ssp}_median.nc'))['ssi'].where(season)
                    spei_data = xr.open_dataset(join(fp, f'SPEI_scale2_{crop}_{ssp}_median.nc'))['spei'].where(season)
                    yied= xr.open_dataset(join(fp, f'yield_{crop}_{ssp}_median.nc'))[f'yield-{crop}-noirr']
                    yied['time'] = ssi_data.time.groupby('time.year').first().values
                    ssi_data=ssi_data[:,:,-len(spei_data.time):]
                    yied=yied[int(-len(spei_data.time)/12):-1]
                    ssi_data = ssi_data.groupby('time.year').mean('time')[:,:,:-1].rename({'year': 'time'})
                    spei_data = spei_data.groupby('time.year').mean('time')[:,:,:-1].rename({'year': 'time'})
                    result_dict1[ssp][crop]['yield'] =yied
                    result_dict1[ssp][crop]['spei'] = spei_data
                    result_dict1[ssp][crop]['ssi'] = ssi_data
                except FileNotFoundError:
                    print(f"Warning: 缺少{crop}数据文件 for {var}-{ssp}")
                    continue


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import xarray as xr
def plot_fig(slope,vmax,vmin,cmap,title,label,save_path):
   
    # Set up figure and projection
    fig = plt.figure(figsize=(6, 3))
    plt.rcParams["font.family"] = "Times New Roman"
    
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([-150, 180, -60, 70], crs=ccrs.PlateCarree())
    
    # Add map features
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5, alpha=0.5)
    ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='lightgray', alpha=0.2)
    ax.add_feature(cfeature.OCEAN, facecolor='white', alpha=0.5)
    
    # Plot trend data
    im = slope.plot(ax=ax, transform=ccrs.PlateCarree(),
                   cmap=cmap, vmax=vmax,vmin=vmin,
                   add_colorbar=False)

    
    # Custom colorbar
    cbar_ax = fig.add_axes([0.15, 0.08, 0.7, 0.02])  # [left, bottom, width, height]
    cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal',
                       extend='both', pad=0.1)
    cbar.set_label(label, fontsize=14)
    cbar.ax.tick_params(labelsize=12)
    
    # Gridlines and frame
    gl = ax.gridlines(draw_labels=False)
    gl.xlines = False
    gl.ylines = False
    # gl.xlabel_style = {'size': 10}
    # gl.ylabel_style = {'size': 10}
    
    # Remove frame
    ax.spines['geo'].set_visible(False)
    # plt.title(title)
    # Adjust layout and save/show
    # plt.tight_layout()
    # if save_path is True:
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
        # plt.savefig(save_path[:-3]+'eps', dpi=300, bbox_inches='tight')
    # plt.tight_layout()
    plt.show()
    

In [ ]:
for crop in crops:
    for ssp in ssps:
    yied=result_dict1[ssp][crop]['yield']
    spei_data=result_dict1[ssp][crop]['spei']
    ssi_data=result_dict1[ssp][crop]['ssi'] 
    
    plot_fig(corr.where(p_value<0.1),1,-1,'BrBG',
             'Probability',f'Correlation (%)',rf'D:\AAUDE\paper\paper8\fig_step2\{crop}_coff_{var1}_{ssp}_max.png')